# Import Library

In [133]:
# Import Library
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

import pandas as pd

### Kata kunci

In [134]:
# Kata Kunci
katakunci = input('Masukkan kata kunci : ')

### Open Selenium Browser

In [135]:
# Get Selenium Browser
chrome_options = Options()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('log-level=2')
driver = webdriver.Chrome(options=chrome_options)

### Search Method sesuai Kata Kunci

In [136]:
# Get Link
def search(katakunci):
    url = 'https://shopee.co.id/search?keyword=' + katakunci
    links = []
    try:
        driver.get(url)
        time.sleep(10)
        driver.execute_script('window.scrollTo(0, 1000);')
        time.sleep(5)
        driver.execute_script('window.scrollTo(1000, 2000);')
        time.sleep(5)
        driver.execute_script('window.scrollTo(2000, 3000);')
        time.sleep(5)
        driver.execute_script('window.scrollTo(3000, 4000);')
        time.sleep(5)
        driver.execute_script('window.scrollTo(4000, 5000);')
        time.sleep(5)
        driver.execute_script('window.scrollTo(5000, 6000);')
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        items = soup.find_all('div', {'class' : 'col-xs-2-4 shopee-search-item-result__item'})
        for item in items:
            link_elem = item.find('a', {'data-sqe' : 'link'})
            if link_elem:
                link = link_elem.get('href')
                links.append(link)
    except TimeoutException:
        print('failed to get links with query')
    return links

### Method Scraping Data

In [137]:
# Get Product Detail
scraped = []
def get_product(produt_url):
    try:
        url = 'https://shopee.co.id' + produt_url
        driver.get(url)
        time.sleep(5)
        
        soup_b = BeautifulSoup(driver.page_source, 'html.parser')

        item_id = soup_b.find('meta', {'property': 'og:url'}).get('content').split('.')[-2]
        id_produk = item_id if item_id is not None else 'ID tidak ditemukan'

        nama_tag = soup_b.find('div', class_='_44qnta').find('span')
        nama = nama_tag.text if nama_tag is not None else 'Nama tidak ditemukan'
        
        harga_tag = soup_b.find('div', class_='pqTWkA')
        harga = harga_tag.text if harga_tag is not None else '0'
        
        rating_tag = soup_b.find('div', class_='_1k47d8 _046PXf')
        rating = rating_tag.text if rating_tag is not None else '0'
        
        terjual_tag = soup_b.find('div', class_='P3CdcB')
        terjual = terjual_tag.text if terjual_tag is not None else '0'
        
        driver.execute_script('window.scrollTo(0, 1500);')
        time.sleep(5)
        
        penjual_tag = soup_b.find('div', class_='VlDReK')
        penjual = penjual_tag.text if penjual_tag is not None else 'Penjual tidak ditemukan'
        
        label = soup_b.find('label', text='Dikirim Dari').find_next_sibling().text.strip()
        lokasi = label if label is not None else 'Lokasi tidak ditemukan'

        desc_tag = soup_b.find('p', class_='irIKAp')
        desc = desc_tag.text if desc_tag is not None else 'Deskripsi tidak ditemukan'

        rating_overview = soup_b.find_all('div', class_='product-rating-overview__filter')
        rating_5 = rating_overview[1].text.split()[2][1:-1]
        rating_4 = rating_overview[2].text.split()[2][1:-1]
        rating_3 = rating_overview[3].text.split()[2][1:-1]
        rating_2 = rating_overview[4].text.split()[2][1:-1]
        rating_1 = rating_overview[5].text.split()[2][1:-1]
        
        print('Scraping ' + id_produk,' - ', nama)
        data_scraped = {
            'kode' : id_produk,
            'nama' : nama, 
            'harga' : harga, 
            'deskripsi' : desc, 
            'penjual' : penjual,
            'lokasi' : lokasi,
            'rating' : rating,
            'rating_5' : rating_5,
            'rating_4' : rating_4,
            'rating_3' : rating_3,
            'rating_2' : rating_2,
            'rating_1' : rating_1,
            'Terjual' : terjual
            }
        scraped.append(data_scraped)
    except TimeoutException:
        print('cant open the link')

In [139]:
products_urls = search(katakunci)

for product_url in products_urls:
    get_product(product_url)

driver.quit()

Scraping 339575556  -  Korset Melahirkan Pants Infrared 3in1 Korset Pelangsing Pengecil Perut Buncit Pakaian Dalam Wanita
Scraping 842867812  -  Kaos Distro bahan combed 24s pria wanita anak Panda Sekolah ori by RAINBOW PANDA
Scraping 47241027  -  Seragam Paskibraka Paskibra Lengkap
Scraping 848135372  -  KAOS WANITA/BAJU CEWEK OVERSIZE KINDNESS MATTERS KAOS BAJU CEWEK KEKINIAN MURAH ATASAN WANITA KATUN (COD)
Scraping 526260429  -  K-MAN CELANA DALAM MAGNET TERAPI VITALITAS PRIA K MAN ORIGINAL KLINK
Scraping 153778078  -  Gamis Dress Busui Katun Rayon Motif Original by Moela [Mone Balmi Marin Zelia Ofcia Blisa Mily Luasi Biloa Mirvi Nalin Rolis Rosci Hinsy Vicia Lavia Beira Sirca Minse Racia Miva]
Scraping 39111161  -  Grosir Solo Gamis Midi Dress Linen Tali Pinggang Hazzel
Scraping 259918782  -  Alinka dress cringkle airflow gamis wanita
Scraping 45229875  -  Hijabwanitacantik - Saqi Dress SQ26.2 Grey | Dress Printing Exclusive | Gamis Pakaian Muslim
Scraping 358383278  -  Gamis Terba

# Make to scrap_data.csv

In [140]:
df1 = pd.DataFrame(scraped)
df1 = df1.to_csv('scrap_data.csv', index = False)